In [1]:
#feauture engineering
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd

In [2]:
df = pd.read_excel("J:\Studies\GUVI\projects\capstone\Zomato\zomato.xlsx")

In [7]:
df1 = df.drop(columns=columns_to_drop, errors='ignore')
df2 = df1.drop(columns=['Average_Cost_For_Two'])
df2

NameError: name 'columns_to_drop' is not defined

In [9]:
# Encode categorical features
label_encoders = {}
categorical_columns = ['Cuisines', 'City', 'Rating_Text']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoder for later use

# Normalize numerical features
scaler = MinMaxScaler()
numerical_columns = ['Average_Cost_For_Two', 'Votes', 'Aggregate_Rating']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print("Feature Engineering Complete:")
df.head()


Feature Engineering Complete:


,Name,Cuisines,Average_Cost_For_Two,Price_Range,Has_Online_Delivery,Is_Delivering_Now,Menu_URL,Booking_URL,Featured_Image,Latitude,Longitude,Address,City,Aggregate_Rating,Rating_Text,Votes,Res_ID
0,Hauz Khas Social,628,0.002000,3,1,0,https://www.zomato.com/HauzKhasSocial/menu?utm...,https://www.zomato.com/HauzKhasSocial/book?utm...,https://b.zmtcdn.com/data/pictures/2/308322/cf...,28.554285,77.194471,"9-A & 12, Hauz Khas Village, New Delhi",89,0.877551,5,0.725352,308322
1,Qubitos - The Terrace Cafe,1812,0.001875,3,0,0,https://www.zomato.com/ncr/qubitos-the-terrace...,https://www.zomato.com/ncr/qubitos-the-terrace...,https://b.zmtcdn.com/data/pictures/7/18037817/...,28.647133,77.117701,"C-7, Vishal Enclave, Opposite Metro Pillar 417...",89,0.918367,1,0.071154,18037817
2,The Hudson Cafe,428,0.001063,2,1,0,https://www.zomato.com/ncr/the-hudson-cafe-del...,NaN,https://b.zmtcdn.com/data/pictures/5/312345/03...,28.694947,77.204317,"2524, 1st Floor, Hudson Lane, Delhi University...",89,0.897959,5,0.140571,312345
3,Summer House Cafe,1046,0.002313,3,0,0,https://www.zomato.com/ncr/summer-house-cafe-h...,https://www.zomato.com/ncr/summer-house-cafe-h...,https://b.zmtcdn.com/data/pictures/0/307490/e0...,28.552520,77.203809,"1st Floor, DDA Shopping Complex, Aurobindo Pla...",89,0.836735,5,0.166728,307490
4,38 Barracks,1475,0.002000,3,0,0,https://www.zomato.com/ncr/38-barracks-connaug...,https://www.zomato.com/ncr/38-barracks-connaug...,https://b.zmtcdn.com/data/pictures/7/18241537/...,28.633025,77.222858,"M-38, Outer Circle, Connaught Place, New Delhi",89,0.897959,5,0.076825,18241537


In [10]:
#model development
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Load the data
print("Loading data...")
df = pd.read_excel("zomato.xlsx")

# Make a copy of the DataFrame
df_cleaned = df.copy()

# Identify columns to keep
columns_to_keep = [
    'Name',  # Keep restaurant name for display
    'Cuisines', 
    'Price_Range', 
    'Has_Online_Delivery',
    'Is_Delivering_Now', 
    'City', 
    'Aggregate_Rating',
    'Rating_Text',
    'Votes',
    'Average_Cost_For_Two'
]

# Keep only the columns we need
df_cleaned = df_cleaned[columns_to_keep]

# Handle missing values if any
df_cleaned = df_cleaned.dropna()

# Create copy for encoding
df_encoded = df_cleaned.copy()

# Initialize scalers
input_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Encode categorical columns
categorical_columns = ['Cuisines', 'City', 'Rating_Text']
label_encoders = {}

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_encoded[col] = label_encoders[col].fit_transform(df_encoded[col].astype(str))

# Scale input numerical features
numerical_features = ['Votes', 'Aggregate_Rating']
df_encoded[numerical_features] = input_scaler.fit_transform(df_encoded[numerical_features])

# Scale target variable
df_encoded['Average_Cost_For_Two'] = target_scaler.fit_transform(df_encoded[['Average_Cost_For_Two']])

# Separate features and target
feature_columns = ['Cuisines', 'Price_Range', 'Has_Online_Delivery', 'Is_Delivering_Now', 
                  'City', 'Aggregate_Rating', 'Rating_Text', 'Votes']
X = df_encoded[feature_columns]
y = df_encoded['Average_Cost_For_Two']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
print("Training model...")
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

# Save all necessary files
print("\nSaving files...")

# Save the original dataframe for reference
df_cleaned.to_pickle('cleaned_data.pkl')

with open('restaurant_cost_prediction_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

with open('input_scaler.pkl', 'wb') as f:
    pickle.dump(input_scaler, f)

with open('target_scaler.pkl', 'wb') as f:
    pickle.dump(target_scaler, f)

with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_columns, f)

print("Files saved successfully!")
print("- cleaned_data.pkl")
print("- restaurant_cost_prediction_model.pkl")
print("- label_encoders.pkl")
print("- input_scaler.pkl")
print("- target_scaler.pkl")
print("- feature_names.pkl")

Loading data...
Training model...

Model Evaluation Metrics:
RMSE: 0.0095
MAE: 0.0003
R² Score: 0.8414

Saving files...
Files saved successfully!
- cleaned_data.pkl
- restaurant_cost_prediction_model.pkl
- label_encoders.pkl
- input_scaler.pkl
- target_scaler.pkl
- feature_names.pkl


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Load the data
df = pd.read_excel("zomato.xlsx")

# Make a copy of the DataFrame
df_cleaned = df.copy()

# Split cuisines and create expanded DataFrame
def expand_cuisines(df):
    # Create rows for each cuisine
    expanded_rows = []
    for _, row in df.iterrows():
        cuisines = str(row['Cuisines']).split(',')
        for cuisine in cuisines:
            new_row = row.copy()
            new_row['Cuisine'] = cuisine.strip()  # Remove whitespace
            expanded_rows.append(new_row)
    
    return pd.DataFrame(expanded_rows)

# Expand the DataFrame
df_expanded = expand_cuisines(df_cleaned)

# Remove the original Cuisines column and rename the new one
df_expanded = df_expanded.drop('Cuisines', axis=1)

# Identify columns to keep
columns_to_keep = [
    'Cuisine',  # New column name
    'Price_Range', 
    'Has_Online_Delivery',
    'Is_Delivering_Now', 
    'City', 
    'Aggregate_Rating',
    'Rating_Text',
    'Votes',
    'Average_Cost_For_Two'
]

# Keep only the columns we need
df_expanded = df_expanded[columns_to_keep]

# Handle missing values if any
df_expanded = df_expanded.dropna()

# Initialize scalers
input_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Encode categorical columns
categorical_columns = ['Cuisine', 'City', 'Rating_Text']
label_encoders = {}

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_expanded[col] = label_encoders[col].fit_transform(df_expanded[col].astype(str))

# Scale input numerical features
numerical_features = ['Votes', 'Aggregate_Rating']
df_expanded[numerical_features] = input_scaler.fit_transform(df_expanded[numerical_features])

# Scale target variable
df_expanded['Average_Cost_For_Two'] = target_scaler.fit_transform(df_expanded[['Average_Cost_For_Two']])

# Separate features and target
X = df_expanded.drop('Average_Cost_For_Two', axis=1)
y = df_expanded['Average_Cost_For_Two']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Transform predictions back to original scale for metrics
y_test_original = target_scaler.inverse_transform(y_test.values.reshape(-1, 1))
y_pred_original = target_scaler.inverse_transform(y_pred.reshape(-1, 1))

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
mae = mean_absolute_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)

print("\nModel Evaluation Metrics:")
print(f"RMSE: ₹{rmse:.2f}")
print(f"MAE: ₹{mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

# Save all necessary files
with open('restaurant_cost_prediction_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

with open('input_scaler.pkl', 'wb') as f:
    pickle.dump(input_scaler, f)

with open('target_scaler.pkl', 'wb') as f:
    pickle.dump(target_scaler, f)

with open('feature_names.pkl', 'wb') as f:
    pickle.dump(list(X.columns), f)

# Also save the original DataFrame for reference
df_expanded.to_pickle('expanded_dataset.pkl')

print("\nSaved files:")
print("- restaurant_cost_prediction_model.pkl")
print("- label_encoders.pkl")
print("- input_scaler.pkl")
print("- target_scaler.pkl")
print("- feature_names.pkl")
print("- expanded_dataset.pkl")


Model Evaluation Metrics:
RMSE: ₹5844.30
MAE: ₹177.39
R² Score: 0.9058

Feature Importance:
               feature    importance
4                 City  9.189104e-01
0              Cuisine  3.369921e-02
7                Votes  2.140749e-02
5     Aggregate_Rating  2.117964e-02
1          Price_Range  3.147533e-03
6          Rating_Text  1.634413e-03
2  Has_Online_Delivery  2.125245e-05
3    Is_Delivering_Now  5.372766e-08

Saved files:
- restaurant_cost_prediction_model.pkl
- label_encoders.pkl
- input_scaler.pkl
- target_scaler.pkl
- feature_names.pkl
- expanded_dataset.pkl


In [11]:
# Make a copy of the DataFrame
df_cleaned = df.copy()

# Encode categorical columns
categorical_columns = ['Cuisines', 'City', 'Rating_Text']  # Add other relevant categorical columns if needed
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df_cleaned[col] = le.fit_transform(df_cleaned[col].astype(str))
    label_encoders[col] = le  # Save encoders for later use

scaler = MinMaxScaler()
numerical_columns = ['Average_Cost_For_Two', 'Votes', 'Aggregate_Rating']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Drop irrelevant columns
columns_to_drop = ['Name', 'Menu_URL', 'Booking_URL', 'Featured_Image', 'Address', 'Latitude', 'Longitude', 'Res_ID']
df_cleaned = df_cleaned.drop(columns=columns_to_drop, errors='ignore')

# Define features and target variable
X = df_cleaned.drop(columns=['Average_Cost_For_Two'])  # Drop target variable
y = df_cleaned['Average_Cost_For_Two']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate RMSE
mse = mean_squared_error(y_test, y_pred)  # Default output is squared
rmse = np.sqrt(mse)  # Take the square root for RMSE

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

print(f"Model Evaluation:\nRMSE: {rmse:.4f}\nMAE: {mae:.4f}")


Model Evaluation:
RMSE: 0.0091
MAE: 0.0003


In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calculate RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R² (coefficient of determination)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Model Evaluation:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

Model Evaluation:
RMSE: 0.0091
MAE: 0.0003
R² Score: 0.8577


In [4]:
import pickle

In [5]:
pickle.format_version

'4.0'

In [13]:
import joblib

In [14]:
# Save the trained model
model_file_path = "restaurant_cost_prediction_model.joblib"
with open(model_file_path, 'wb') as file:
    joblib.dump(model, file)

# Save the LabelEncoders
encoder_file_path = "label_encoders.joblib"
with open(encoder_file_path, 'wb') as file:
    joblib.dump(label_encoders, file)

# Save feature names
feature_names = list(X.columns)  # X is your training dataframe
with open("feature_names.joblib", "wb") as f:
    joblib.dump(feature_names, f)

scaler_file_path = "minmax_scaler.joblib"
with open(scaler_file_path, "wb") as file:
    joblib.dump(scaler, file)

print(f"Model and encoders saved:\nModel Path: {model_file_path}\nEncoders Path: {encoder_file_path}")

Model and encoders saved:
Model Path: restaurant_cost_prediction_model.joblib
Encoders Path: label_encoders.joblib


In [22]:
# Save the trained model
model_file_path = "restaurant_cost_prediction_model.pkl"
with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)

# Save the LabelEncoders
encoder_file_path = "label_encoders.pkl"
with open(encoder_file_path, 'wb') as file:
    pickle.dump(label_encoders, file)

# Save feature names
feature_names = list(X.columns)  # X is your training dataframe
with open("feature_names.pkl", "wb") as f:
    pickle.dump(feature_names, f)

scaler_file_path = "minmax_scaler.pkl"
with open(scaler_file_path, "wb") as file:
    pickle.dump(scaler, file)

print(f"Model and encoders saved:\nModel Path: {model_file_path}\nEncoders Path: {encoder_file_path}")


Model and encoders saved:
Model Path: restaurant_cost_prediction_model.pkl
Encoders Path: label_encoders.pkl


In [13]:
import pandas as pd
import pickle
import numpy as np

# Load the model and encoders
model_file_path = "restaurant_cost_prediction_model.pkl"  # Replace with your file path
encoder_file_path = "label_encoders.pkl"
feature_names_file_path = "feature_names.pkl"

with open(model_file_path, 'rb') as file:
    model = pickle.load(file)

with open(encoder_file_path, 'rb') as file:
    label_encoders = pickle.load(file)

with open(feature_names_file_path, "rb") as file:
    trained_features = pickle.load(file)

# Load dataset for visualization (if needed)
data_file_path = "J:\\Studies\\GUVI\\projects\\capstone\\Zomato\\zomato.xlsx"  # Replace with your file path
df = pd.read_excel(data_file_path)

# Input restaurant details
cuisine_input = input("Enter Cuisine: ")
rating_text_input = input("Enter Rating Text: ")
votes_input = int(input("Enter Votes: "))
selected_city = input("Enter City: ")

# Add fallback categories to LabelEncoder if needed
def ensure_fallback_class(label_encoder, fallback_value):
    if fallback_value not in label_encoder.classes_:
        label_encoder.classes_ = np.append(label_encoder.classes_, fallback_value)

# Ensure fallback values are added
ensure_fallback_class(label_encoders['Cuisines'], "Other")
ensure_fallback_class(label_encoders['Rating_Text'], "Good")
ensure_fallback_class(label_encoders['City'], "Other")  # Ensure fallback for City as well

# Preprocess cuisine input: Take the first cuisine if it’s a comma-separated string
def preprocess_cuisine(cuisine):
    if isinstance(cuisine, str) and ',' in cuisine:
        return cuisine.split(',')[0].strip()  # Take the first cuisine
    return cuisine.strip() if isinstance(cuisine, str) else "Other"

# Encode with fallback
def safe_encode(label_encoder, value, fallback_value):
    try:
        if value in label_encoder.classes_:
            return label_encoder.transform([value])[0]
        else:
            print(f"Unseen value: '{value}'. Using fallback: '{fallback_value}'.")
            return label_encoder.transform([fallback_value])[0]
    except Exception as e:
        print(f"Encoding error: {e}")
        raise

# Preprocess inputs
processed_cuisine_input = preprocess_cuisine(cuisine_input)
encoded_cuisine = safe_encode(label_encoders['Cuisines'], processed_cuisine_input, "Other")
encoded_rating_text = safe_encode(label_encoders['Rating_Text'], rating_text_input, "Good")
encoded_city = safe_encode(label_encoders['City'], selected_city, "Other")  # Encode the city

# Add all required features for prediction
default_values = {
    "Aggregate_Rating": 3.5,  # Default average rating
    "Has_Online_Delivery": 0,  # Assuming default is 'No' (0)
    "Is_Delivering_Now": 0,  # Assuming default is 'No' (0)
    "Price_Range": 2,  # Assuming mid-range price as default
}

input_features = {
    'Cuisines': encoded_cuisine,
    'Price_Range': default_values['Price_Range'],
    'Has_Online_Delivery': default_values['Has_Online_Delivery'],
    'Is_Delivering_Now': default_values['Is_Delivering_Now'],
    'City': encoded_city,  # Encoded city
    'Aggregate_Rating': default_values['Aggregate_Rating'],
    'Rating_Text': encoded_rating_text,
    'Votes': votes_input,
}

# Convert input dictionary to DataFrame
input_df = pd.DataFrame([input_features])

# Match the column order used during model training
expected_columns = ['Cuisines', 'Price_Range', 'Has_Online_Delivery', 
                    'Is_Delivering_Now', 'City', 'Aggregate_Rating', 
                    'Rating_Text', 'Votes']
input_df = input_df[expected_columns]

# Make prediction
try:
    prediction = model.predict(input_df)[0]
    print(f"The predicted average cost for two is: ₹{prediction:.2f}")
except Exception as e:
    print(f"Prediction error: {e}")


Unseen value: 'Cafe'. Using fallback: 'Other'.
Unseen value: 'Very Good'. Using fallback: 'Good'.
Unseen value: 'New Delhi'. Using fallback: 'Other'.
The predicted average cost for two is: ₹0.00


In [27]:
import pandas as pd
import pickle
import numpy as np

# Load the model, encoders, feature names, and scaler
model_file_path = "restaurant_cost_prediction_model.pkl"  # Replace with your file path
encoder_file_path = "label_encoders.pkl"
feature_names_file_path = "feature_names.pkl"
scaler_file_path = "minmax_scaler.pkl"  # File path for the saved MinMaxScaler

with open(model_file_path, 'rb') as file:
    model = pickle.load(file)

with open(encoder_file_path, 'rb') as file:
    label_encoders = pickle.load(file)

with open(feature_names_file_path, "rb") as file:
    trained_features = pickle.load(file)

with open(scaler_file_path, "rb") as file:
    scaler = pickle.load(file)

# Input restaurant details
cuisine_input = input("Enter Cuisine: ")
rating_text_input = input("Enter Rating Text: ")
votes_input = int(input("Enter Votes: "))
selected_city = input("Enter City: ")

# Add fallback categories to LabelEncoder if needed
def ensure_fallback_class(label_encoder, fallback_value):
    if fallback_value not in label_encoder.classes_:
        label_encoder.classes_ = np.append(label_encoder.classes_, fallback_value)

# Ensure fallback values are added
ensure_fallback_class(label_encoders['Cuisines'], "Other")
ensure_fallback_class(label_encoders['Rating_Text'], "Good")
ensure_fallback_class(label_encoders['City'], "Other")  # Ensure fallback for City as well

# Preprocess cuisine input: Take the first cuisine if it’s a comma-separated string
def preprocess_cuisine(cuisine):
    if isinstance(cuisine, str) and ',' in cuisine:
        return cuisine.split(',')[0].strip()  # Take the first cuisine
    return cuisine.strip() if isinstance(cuisine, str) else "Other"

# Encode with fallback
def safe_encode(label_encoder, value, fallback_value):
    try:
        if value in label_encoder.classes_:
            return label_encoder.transform([value])[0]
        else:
            print(f"Unseen value: '{value}'. Using fallback: '{fallback_value}'.")
            return label_encoder.transform([fallback_value])[0]
    except Exception as e:
        print(f"Encoding error: {e}")
        raise

# Preprocess inputs
processed_cuisine_input = preprocess_cuisine(cuisine_input)
encoded_cuisine = safe_encode(label_encoders['Cuisines'], processed_cuisine_input, "Other")
encoded_rating_text = safe_encode(label_encoders['Rating_Text'], rating_text_input, "Good")
encoded_city = safe_encode(label_encoders['City'], selected_city, "Other")  # Encode the city

# Add all required features for prediction
default_values = {
    "Aggregate_Rating": 3.5,  # Default average rating
    "Has_Online_Delivery": 0,  # Assuming default is 'No' (0)
    "Is_Delivering_Now": 0,  # Assuming default is 'No' (0)
    "Price_Range": 2,  # Assuming mid-range price as default
}

input_features = {
    'Cuisines': encoded_cuisine,
    'Price_Range': default_values['Price_Range'],
    'Has_Online_Delivery': default_values['Has_Online_Delivery'],
    'Is_Delivering_Now': default_values['Is_Delivering_Now'],
    'City': encoded_city,  # Encoded city
    'Aggregate_Rating': default_values['Aggregate_Rating'],
    'Rating_Text': encoded_rating_text,
    'Votes': votes_input,
}

# Convert input dictionary to DataFrame
input_df = pd.DataFrame([input_features])

# Match the column order used during scaler fitting and model training
input_df = input_df.reindex(columns=trained_features, fill_value=0)

# Scale the input features using the loaded MinMaxScaler
try:
    scaled_input_df = scaler.transform(input_df)
    # Make prediction
    prediction = model.predict(scaled_input_df)[0]
    print(f"The predicted average cost for two is: ₹{prediction:.2f}")
except ValueError as e:
    print(f"Scaler error: {e}")
    print(f"Input columns: {input_df.columns}")
    print(f"Expected columns: {trained_features}")
    raise
except Exception as e:
    print(f"Prediction error: {e}")



Unseen value: 'Continental'. Using fallback: 'Other'.
Unseen value: 'Very Good'. Using fallback: 'Good'.
Unseen value: 'New Delhi'. Using fallback: 'Other'.
Scaler error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- City
- Cuisines
- Has_Online_Delivery
- Is_Delivering_Now
- Price_Range
- ...
Feature names seen at fit time, yet now missing:
- Average_Cost_For_Two

Input columns: Index(['Cuisines', 'Price_Range', 'Has_Online_Delivery', 'Is_Delivering_Now',
       'City', 'Aggregate_Rating', 'Rating_Text', 'Votes'],
      dtype='object')
Expected columns: ['Cuisines', 'Price_Range', 'Has_Online_Delivery', 'Is_Delivering_Now', 'City', 'Aggregate_Rating', 'Rating_Text', 'Votes']


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- City
- Cuisines
- Has_Online_Delivery
- Is_Delivering_Now
- Price_Range
- ...
Feature names seen at fit time, yet now missing:
- Average_Cost_For_Two


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Load the data
df = pd.read_excel("zomato.xlsx")

# Make a copy of the DataFrame
df_cleaned = df.copy()

# Identify columns to keep
columns_to_keep = [
    'Cuisines', 
    'Price_Range', 
    'Has_Online_Delivery',
    'Is_Delivering_Now', 
    'City', 
    'Aggregate_Rating',
    'Rating_Text',
    'Votes',
    'Average_Cost_For_Two'
]

# Keep only the columns we need
df_cleaned = df_cleaned[columns_to_keep]

# Handle missing values if any
df_cleaned = df_cleaned.dropna()

# Initialize scalers
input_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Encode categorical columns
categorical_columns = ['Cuisines', 'City', 'Rating_Text']
label_encoders = {}

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_cleaned[col] = label_encoders[col].fit_transform(df_cleaned[col].astype(str))

# Scale input numerical features
numerical_features = ['Votes', 'Aggregate_Rating']
df_cleaned[numerical_features] = input_scaler.fit_transform(df_cleaned[numerical_features])

# Scale target variable
df_cleaned['Average_Cost_For_Two'] = target_scaler.fit_transform(df_cleaned[['Average_Cost_For_Two']])

# Separate features and target
X = df_cleaned.drop('Average_Cost_For_Two', axis=1)
y = df_cleaned['Average_Cost_For_Two']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Transform predictions back to original scale for metrics
y_test_original = target_scaler.inverse_transform(y_test.values.reshape(-1, 1))
y_pred_original = target_scaler.inverse_transform(y_pred.reshape(-1, 1))

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
mae = mean_absolute_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)

print("\nModel Evaluation Metrics:")
print(f"RMSE: ₹{rmse:.2f}")
print(f"MAE: ₹{mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

# Save all necessary files
with open('restaurant_cost_prediction_model.joblib', 'wb') as f:
   joblib.dump(model, f)

with open('label_encoders.joblib', 'wb') as f:
    joblib.dump(label_encoders, f)

with open('input_scaler.joblib', 'wb') as f:
    joblib.dump(input_scaler, f)

with open('target_scaler.joblib', 'wb') as f:
    joblib.dump(target_scaler, f)

with open('feature_names.joblib', 'wb') as f:
    joblib.dump(list(X.columns), f)

print("\nSaved files:")
print("- restaurant_cost_prediction_model.joblib")
print("- label_encoders.joblib")
print("- input_scaler.joblib")
print("- target_scaler.joblib")
print("- feature_names.joblib")


Model Evaluation Metrics:
RMSE: ₹7605.98
MAE: ₹256.50
R² Score: 0.8414

Feature Importance:
               feature    importance
4                 City  9.415408e-01
0             Cuisines  3.243716e-02
7                Votes  1.265464e-02
5     Aggregate_Rating  1.144023e-02
1          Price_Range  1.696090e-03
6          Rating_Text  1.999268e-04
2  Has_Online_Delivery  3.108540e-05
3    Is_Delivering_Now  2.704755e-08

Saved files:
- restaurant_cost_prediction_model.joblib
- label_encoders.joblib
- input_scaler.joblib
- target_scaler.joblib
- feature_names.joblib


In [17]:
pip show joblib

Name: joblib
Version: 1.4.2
Summary: Lightweight pipelining with Python functions
Home-page: https://joblib.readthedocs.io
Author: 
Author-email: Gael Varoquaux <gael.varoquaux@normalesup.org>
License: BSD 3-Clause
Location: c:\users\joshua sam\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: scikit-learn
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import pickle

def load_preprocessors():
    with open('restaurant_cost_prediction_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('label_encoders.pkl', 'rb') as f:
        encoders = pickle.load(f)
    with open('input_scaler.pkl', 'rb') as f:
        input_scaler = pickle.load(f)
    with open('target_scaler.pkl', 'rb') as f:
        target_scaler = pickle.load(f)
    with open('feature_names.pkl', 'rb') as f:
        feature_names = pickle.load(f)
    return model, encoders, input_scaler, target_scaler, feature_names

def predict_price():
    # Load all preprocessors
    model, encoders, input_scaler, target_scaler, feature_names = load_preprocessors()
    
    try:
        # Get user input
        print("\nEnter restaurant details:")
        cuisine = input("Cuisine (e.g., North Indian, Chinese): ")
        city = input("City (e.g., New Delhi): ")
        rating = input("Rating Text (Good/Very Good/Excellent): ")
        votes = int(input("Number of Votes: "))
        
        # Create input data dictionary
        input_data = {
            'Cuisines': cuisine,
            'Price_Range': 2,  # Default to mid-range
            'Has_Online_Delivery': 0,  # Default values
            'Is_Delivering_Now': 0,    # Default values
            'City': city,
            'Aggregate_Rating': 3.5,  # Default to average rating
            'Rating_Text': rating,
            'Votes': votes
        }
        
        # Create DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Encode categorical variables
        for col in ['Cuisines', 'City', 'Rating_Text']:
            try:
                input_df[col] = encoders[col].transform(input_df[col].astype(str))
            except:
                # Handle unknown categories
                print(f"Warning: Unknown {col} category. Using a default value.")
                input_df[col] = encoders[col].transform([encoders[col].classes_[0]])
        
        # Scale numerical features
        numerical_features = ['Votes', 'Aggregate_Rating']
        input_df[numerical_features] = input_scaler.transform(input_df[numerical_features])
        
        # Ensure correct column order
        input_df = input_df[feature_names]
        
        # Make prediction (scaled)
        scaled_prediction = model.predict(input_df)[0]
        
        # Transform prediction back to original scale
        prediction = target_scaler.inverse_transform([[scaled_prediction]])[0][0]
        
        print(f"\nPredicted average cost for two people: ₹{prediction:.2f}")
        
        # Print additional information
        if prediction <= 500:
            price_category = "Budget-friendly"
        elif prediction <= 1500:
            price_category = "Mid-range"
        else:
            price_category = "High-end"
            
        print(f"Price Category: {price_category}")
        
    except ValueError as ve:
        print(f"Invalid input: {ve}")
        print("Please ensure you enter a valid number for votes.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please ensure you have run the training script first and all .pkl files are in the current directory.")

if __name__ == "__main__":
    predict_price()


Enter restaurant details:

Predicted average cost for two people: ₹869.13
Price Category: Mid-range
